<a href="https://colab.research.google.com/github/pedroh6s/Conhecendo-o-PANDAS/blob/main/Roteiro_projeto_b%C3%A1sico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTAÇÃO DAS BIBLIOTECAS**

In [60]:
import pandas as pd
import pandera as pa

Extração dos dados de uma fonte ou fontes.

É possível e importante já fazer alguns tratamentos básicos nos dados durante a conversão, como transformar os tipos de dados de cada coluna.

Nesse caso realizou-se a conversão da coluna de datas e além do tipo de dados, convertou-se também o padrão da data pra o padrão brasileiro. 

In [31]:
df = pd.read_csv("/content/drive/MyDrive/FONTES DE DADOS/ocorrencia_2010_2020 - ocorrencia_2010_2020.csv", sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True)
backup = df[:]

In [32]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

O comando **pd.unique(df['nome da coluna'])** retorna um lista contendo todos os valores únicos para a coluna escolhida.

Também pode ser escrito assim **df['nome da coluna'].unique()**

Além da lista, a função também retorna o tipo de dados da coluna.

Como o output dessa função é uma lista, é possível usar métodos de listas paa tratar o output e conseguir novas informações como tamanho, ordem alfabética, etc.

É possível colocar o comando unique dentro do comando **sorted()** porém o resultado não trará mais o tipo de dados, uma vez que essa informação está fora da lista que o comando retorna.

Usando o comando sorted o resultado será apenas a lista de valores alfabeticamente.


In [33]:
pd.unique(df['ocorrencia_uf'])

array(['RJ', 'PA', 'MT', 'RS', 'BA', 'AM', 'PR', 'MG', 'SP', 'DF', 'AC',
       'TO', 'SC', 'GO', 'MA', 'ES', 'PE', 'RR', 'MS', 'CE', 'AP', 'AL',
       'RO', 'SE', 'PB', 'PI', 'RN', '**'], dtype=object)

É interessante anotar as inconsistências ou prováveis inconsistências encontradas para em seguida analisá-las.

Isso garante a integridade dos dados.


Nesse dataframe foram encontradas as seguintes inconsistências.

Coluna ocorrencia_uf: 
```
"**"
```

COluna ocorrencia_aerodromo:

```
 '****', '*****', '####', '###!'
```




In [ ]:
df.loc[df.ocorrencia_uf == '**']

In [35]:
df.replace(['**', '****', '*****', '####', '###!'], pd.NA, inplace=True)

É preferível ter um dado nulo do que um dado sujo.

In [36]:
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [41]:
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


O método **dropna** retira todas as linhas com resultado <NA>, portanto resultados nulos.

**df.dropna()**

OBS: para executar as mudanças é necessário informar o parâmetro **inplace=True**

Isso resulta em uma perda de dados.

É necessário analisar se é melhor ter dados incompletos, ou dados a menos.

No caso desse banco de dados específico seriam perdidas mais de 2 mil linhas, o que é um percentual altíssimo que muito provavelmente alteraria consideravelmente essa análise.

In [40]:
df.dropna()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
...,...,...,...,...,...,...,...,...,...
5743,79824,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00,0
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


A função **dropna** pode ser usada para deletar subsets específicos, ou seja, podemos escolher em qual ou quais colunas procuras os valores desejados e deletar apenas as linhas que possuem aquele valor específico em uma certa coluna.

**df.dropna(subset=['nome da coluna'])**

In [ ]:
df.dropna(['ocorrencia_uf'])

A função **drop_duplicates** vai procurar registros duplicados, ou seja, quando todos os valores da linha são iguais e caso encontre vai manter apenas a primeira ocorrência daquele registro.

**df.drop_duplicates()**

**OBS:** Para executar as mudanças é necessário o parâmetro **inplace=True**


In [ ]:
df.drop_duplicates()

**VALIDAÇÃO DE DADOS**

Devemos informar o tipo de dados esperado para ser informado em cada coluna do seu dataframe.

O tipo object é informado como string.

Podemos verificar a quantidade de dígitos em uma coluna usando os parâmetros:

**pa.Check.str_length(tamanho_mínimo, tamanho_máximo)**

In [73]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String,pa.Check.str_length(2.2))
    }
)

Se o dataframe for exibido com o comando **schema.validate(df)** significa que o esquema foi validado. 

Caso você tenha definido os tipos de dados de maneiro equivocada, você receberá um erro como por exemplo:

 **SchemaError: expected series 'codigo_ocorrencia' to have type str, got int64**


In [ ]:
schema.validate(df)